# Проект "Исследование рынка заведений общественного питания Москвы" #
Цель проекта:
- на основании исследования рынка заведений общественного питания в Москве, подготовить презентацию для привлечения инвестиций в открытие кафе в Москве

# Cодержание #

<a id='data_description'></a>
# Описание данных #
Таблица rest_data:
id — идентификатор объекта;
object_name — название объекта общественного питания;
chain — сетевой ресторан;
object_type — тип объекта общественного питания;
address — адрес;
number — количество посадочных мест.

<a id='analyse_plan'></a>
# План проведения исследования #
1. загрузка данных
2. подготовка данных
3. анализ данных

<a id='import'></a>
# Импорт библиотек #

In [1]:
import io

import pandas as pd
from IPython.display import display
import numpy as np
from scipy import stats as st
import plotly_express as px
import os
import warnings
import requests

pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

<a id='data'></a>
# Загрузка данных #

In [2]:
path_1 = 'datasets/'
path_2 = '/datasets/'
rest_data = 'rest_data.csv'
street_reg_data = 'mosgaz-streets.csv'
path_rest = ''
path_street_reg = ''

if os.path.exists(path_1):
    path_rest = path_1 + rest_data
    path_street_reg = path_1 + street_reg_data
elif os.path.exists(path_2):
    path_rest = path_2 + rest_data
    path_street_reg = path_2 + street_reg_data
else: print('Данные отсутствуют. Проверьте путь к папкам с данными')

data = pd.read_csv(filepath_or_buffer=path_rest)
data_street_reg = pd.read_csv(filepath_or_buffer=path_street_reg)

In [3]:
data_street_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   streetname  4398 non-null   object
 1   areaid      4398 non-null   int64 
 2   okrug       4398 non-null   object
 3   area        4398 non-null   object
dtypes: int64(1), object(3)
memory usage: 137.6+ KB


In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15366 non-null  int64 
 1   object_name  15366 non-null  object
 2   chain        15366 non-null  object
 3   object_type  15366 non-null  object
 4   address      15366 non-null  object
 5   number       15366 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 720.4+ KB
None


In [5]:
print(data.chain.unique())
print(data.object_type.unique())

['нет' 'да']
['кафе' 'столовая' 'закусочная' 'предприятие быстрого обслуживания'
 'ресторан' 'кафетерий' 'буфет' 'бар' 'магазин (отдел кулинарии)']


## Итоги предварительного осмотра данных ##
1. data['object_name']:
   - перевести данные в нижний регистр
2. data['chain'].dtype(object) -> data['chain'].dtype(category)
3. data['object_type'].dtype(object) -> data['object_type'].dtype(category)
4. data['address']:
   - перевести в нижний регистр
   - выделить отдельный столбец с названием улицы
5. перевести в нижний регистр значения:
   - data_street_reg['streetname']
   - data_street_reg['okrug']
   - data_street_reg['area']

<a id='prepare'></a>
# Предобработка данных #

<a id='prepare_change'></a>
## Изменение данных ##

In [6]:
data['object_name'] = data['object_name'].str.strip().str.lower()
data['address'] = data['address'].str.strip().str.lower()
data_street_reg['streetname'] = data_street_reg['streetname'].str.strip().str.lower()
data_street_reg['okrug'] = data_street_reg['okrug'].str.strip().str.lower()
data_street_reg['area'] = data_street_reg['area'].str.strip().str.lower()

<a id='prepare_delete_data'></a>
## Удаление данных ##

In [7]:
print(data.shape)
include_words = ['поселение','город зеленоград','город московский','посёлок','деревня','город щербинка','город троицк']
data = data.drop(index=data[data.stack().str.contains('|'.join(include_words)).any(level=0)].index)
print(data.shape)

(15366, 6)
(14498, 6)


<a id='prepare_type'></a>
## Изменение типов данных ##

In [8]:
data['chain'] = data['chain'].replace(to_replace=['да', 'нет'], value=['сетевое', 'несетевое']).astype('category')
data['object_type'] = data['object_type'].astype('category')

<a id='prepare_street'></a>
## Создание нового столбца ##

In [9]:
data['street'] = data['address'].apply(lambda x: str.split(x, ',')[1].strip() if 'город' in x else str.split(x, ',')[0].strip())

<a id='prepare_dup'></a>
## Поиск дубликатов/пропусков данных ##

In [10]:
print(data.isna().sum())
data = data.drop_duplicates()
print(data.duplicated().sum())

id             0
object_name    0
chain          0
object_type    0
address        0
number         0
street         0
dtype: int64
0


## Итоги предобработки данных ##
1. переведены в нижний регистр
   - data['object_name']
   - data['address']
2. изменен тип данных:
   - data['chain']
   - data['object_type']
3. создан новый столбец:
   - data['street'] - улица города

<a id='analyse'></a>
# Анализ данных #

<a id='analyse_object_kind'></a>
## Соотношение видов объектов общественного питания ##

In [11]:
fig = px.pie(data_frame=data.groupby(by='chain', axis='index')\
                            .agg({'id':'count'})\
                            .reset_index()\
                            .rename(columns={'chain':'type',
                                             'id':'count'}),
             names='type',
             values='count',
             template='seaborn',
             title='Отношение количества сетевых/несетевых заведений',
             width=1000, height=600,)
fig.update_layout(legend_title='Вид заведения')
fig.show()


## Итоги осмотра соотношения видов заведений ##
1. на рынке общественного питания Москвы преобладают несетевые заведения.
   Доли распределены следующим образом:
   - несетевые заведения - 80.6%(12050 шт.)
   - сетевые заведения - 19.4%(2896 шт.)

## Соотношение типов объектов общественного питания ##

In [12]:
fig = px.pie(data_frame=data.groupby(by='object_type', axis='index')\
                            .agg({'id':'count'})\
                            .reset_index()\
                            .rename(columns={'id':'count'}),
             names='object_type',
             values='count',
             template='seaborn',
             title='Соотношение типов заведений',
             width=1000, height=600,)
fig.update_layout(legend_title = 'Тип заведения')
fig.show()

## Итоги осмотра соотношения типов объектов общественного питания ##
1. 83.8% объектов общественного питания составляют следующие типы:
   - кафе - 39.4%(5711 шт.)
   - столовая - 16.9%(2447 шт.)
   - ресторан - 15.1%(2185 шт.)
   - предприятие быстрого обслуживания - 12.5%(1819 шт.)
2. оставшуюся долю - 16.2% составляют следующие типы
   - бар
   - буфет
   - кафетерий
   - закусочная
   - магазин(отдел кулинарии)

<a id='analyse_kind_of_type'></a>
## Виды заведений характерные для типов заведений ##

In [13]:
fig = px.bar(data_frame=data.pivot_table(index=['object_type', 'chain'],
                                         values=['id'],
                                         aggfunc='count').reset_index(),
       x='object_type',
       y='id',
       color='chain',
       barmode='group',
       template='seaborn',
       text_auto=True,
       labels={'id':'Кол-во заведений',
               'object_type':'Тип заведения',
               'chain':'Вид заведения'},
       title='Виды заведений характерные для типов заведений',
             width=1000, height=600,)
fig.show()

## Итоги осмотра характерных видов заведений для типов заведений ##
1. Сетевой вид заведения наиболее характерен для следующих типов заведений:
   - кафе
   - предприятие быстрого обслуживания
   - ресторан

<a id='analyse_net_seat'></a>
## Характеристика сетевых заведений по количеству посадочных мест ##

In [14]:
centroid = data['number'].mean()

In [15]:
data['seat_type'] = data['number'].apply(lambda x: f'много(>={centroid.round()})' if x >= centroid else f'мало(<{centroid.round()})')
data['seat_type'] = data['seat_type'].astype('category')

In [16]:
fig = px.pie(data_frame=data.query('chain == "сетевое"')
                            .groupby(by='seat_type')
                            .agg({'id':'count'})
                            .reset_index()
                            .rename(columns={'id':'count'}),
             names='seat_type',
             values='count',
             template='seaborn',
             title='Характеристика сетевых заведений по количеству посадочных мест',
             width=1000, height=600,)
fig.update_layout(legend_title = 'Кол-во мест')
fig.show()

## Итоги осмотра характеристики сетевых заведений по количеству посадочных мест ##
Сетевые заведения предпочитают небольшие залы, с количеством посадочных мест <60.

<a id='analyse_seat_type'></a>
## Среднее количество мест характерное для типа заведения ##

In [17]:
fig = px.bar(data_frame=data.groupby(by='object_type')
                            .agg({'number':np.mean})
                            .reset_index()
                            .sort_values(by='number', ascending=False),
             x='object_type',
             y='number',
             template='seaborn',
             text_auto='3.0f',
             color='object_type',
             title='Среднее количество мест характерное для типа заведения',
             width=1000, height=600,
             labels={'number':'Ср. кол-во мест',
                     'object_type':'Тип заведения'})

fig.show()

## Итоги осмотра среднего количества мест характерного для типа заведения ##
1. Будем ориентироваться на заведения типа "кафе" и близкие к ним:
   - кафе ~41 посадочных мест
   - бар ~44 посадочных места
   - ресторан ~98 посадочных мест
2. Самое большое среднее количество посадочных мест предоставляет:
   - столовая

<a id='analyze_top_street'></a>
## Улицы с наибольшим количеством заведений ##

In [18]:
street_top = data.groupby(by=['street'], as_index=False)\
                 .agg({'id':'count'})\
                 .rename(columns = {'id':'count'})\
                 .sort_values(by='count', ascending=False)\
                 .head(10)

fig = px.bar(data_frame=street_top,
             x='street',
             y='count',
             color='street',
             template='seaborn',
             text_auto='3.0i',
             title='Улицы с наибольшим количеством заведений',
             width=1000, height=600,
             labels={'count':'Кол-во заведений',
                     'street':'Название улицы'})
fig.show()

<a id='analyze_low_area'></a>
## Районы с большим количеством заведений ##

In [27]:
area_top = street_top.merge(data_street_reg[['streetname', 'area']],
                            left_on='street',
                            right_on='streetname')\
                     .drop(columns='streetname')\
                     .groupby(by='area', as_index=False)\
                     .agg({'count':'sum'})\
                     .sort_values(by='count', ascending=False).head(10)

fig = px.bar(data_frame=area_top,
             x='area',
             y='count',
             color='area',
             template='seaborn',
             text_auto='3.0i',
             title='Районы с наибольшим количеством заведений',
             width=1000, height=600,
             labels={'count':'Кол-во заведений',
                     'area':'Название района'})
fig.show()

<a id='analyze_low_area'></a>
## Улицы/районы с одним заведением ##

In [20]:
area_low = data.groupby(by=['street'], as_index=False)\
               .agg({'id':'count'})\
               .rename(columns = {'id':'count'})\
               .query('count == 1')\
               .merge(data_street_reg[['streetname', 'area']],
                      left_on='street',
                      right_on='streetname',
                      how='left')\
                      .drop(columns='streetname')\
                      .groupby(by='area', as_index=False)\
                      .agg({'count':'count'})\
                      .query('count == 1')
display(area_low)

,area,count
1,алтуфьевский район,1
15,молжаниновский район,1
18,обручевский район,1
28,район восточный,1
30,район гагаринский,1
34,район западное дегунино,1
36,район ивановское,1
42,район крюково,1
46,район левобережный,1
48,район лианозово,1


<a id='prepare_add_region'></a>
## Добавление районов ##

In [21]:
data = data.merge(data_street_reg[['streetname', 'area']],
                  left_on='street',
                  right_on='streetname',
                  how='left')

data = data.drop(columns='streetname',
                 index=data[data['area'].isna()].index)

data.shape

(28997, 9)

In [22]:
data

,id,object_name,chain,object_type,address,number,street,seat_type,area
0,151635,сметана,несетевое,кафе,"город москва, улица егора абакумова, дом 9",48,улица егора абакумова,мало(<60.0),ярославский район
1,77874,родник,несетевое,кафе,"город москва, улица талалихина, дом 2/1, корпус 1",35,улица талалихина,мало(<60.0),таганский район
2,77874,родник,несетевое,кафе,"город москва, улица талалихина, дом 2/1, корпус 1",35,улица талалихина,мало(<60.0),нижегородский район
3,77874,родник,несетевое,кафе,"город москва, улица талалихина, дом 2/1, корпус 1",35,улица талалихина,мало(<60.0),район южное бутово
4,24309,кафе «академия»,несетевое,кафе,"город москва, абельмановская улица, дом 6",95,абельмановская улица,много(>=60.0),таганский район
...,...,...,...,...,...,...,...,...,...
29136,209264,шоколадница,сетевое,кафе,"город москва, улица земляной вал, дом 33",10,улица земляной вал,мало(<60.0),таганский район
29137,209264,шоколадница,сетевое,кафе,"город москва, улица земляной вал, дом 33",10,улица земляной вал,мало(<60.0),басманный район
29138,209186,шоколадница,сетевое,кафе,"город москва, улица земляной вал, дом 33",20,улица земляной вал,мало(<60.0),таганский район
29139,209186,шоколадница,сетевое,кафе,"город москва, улица земляной вал, дом 33",20,улица земляной вал,мало(<60.0),басманный район


<a id='analyse_map'></a>
## Распределение заведений по территории Москвы ##

In [23]:
# data_district = pd.read_csv(filepath_or_buffer='datasets/data_part_district.csv').drop(columns='Unnamed: 0')

In [24]:
# fig = px.pie()
# display(data_district.groupby(by=['district'], as_index=False)
#                      .agg({'id':'count'})
#                      .rename(columns = {'id':'count'})
#                      .sort_values(by='count', ascending=False)
#                      .head(10))